# Photometry Homework

In [111]:
from astropy.io import fits
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

In [114]:
darks_filenames = glob('2020-10-17/h_persei_darks*')
darks = []
darks_filters = []

for i in range(len(darks_filenames[1:])):
    darks.append(fits.getdata(darks_filenames[i]))
    hdul = fits.open(darks_filenames[i])
    darks_filters.append(hdul[0].header['FILTER'])
    hdul.close()
    
darks = np.asarray(darks)
darks = darks.astype(float)

In [115]:
flats_filenames = glob('2020-10-17/h_persei_flats*')
flats = []
flats_filters = []

for i in range(len(flats_filenames[1:])):
    flats.append(fits.getdata(flats_filenames[i]))
    hdul = fits.open(flats_filenames[i])
    flats_filters.append(hdul[0].header['FILTER'])
    hdul.close()
    

flats_filters = np.asarray(flats_filters)
flats = np.asarray(flats)
flats = flats.astype(float)

R_flats = flats[np.where(flats_filters == 'R')]
V_flats = flats[np.where(flats_filters == 'V')]
B_flats = flats[np.where(flats_filters == 'B')]
I_flats = flats[np.where(flats_filters == 'I')]

In [116]:
seq_filenames = np.sort(glob('2020-10-17/h_persei_seq*'))
seq_light = []
seq_dark = []
seq_filters = []

for i in range(len(seq_filenames[1:])):
        
    hdul = fits.open(seq_filenames[i])
    if hdul[0].header['FRAME'] == 'Light':
        seq_filters.append(hdul[0].header['FILTER'])
        seq_light.append(fits.getdata(seq_filenames[i]))
    else:
        seq_dark.append(fits.getdata(seq_filenames[i]))
    hdul.close()

seq_filters = np.asarray(seq_filters)
seq_light = np.asarray(seq_light)
seq_light = seq_light.astype(float)


R_seq = seq_light[np.where(seq_filters == 'R')]
V_seq = seq_light[np.where(seq_filters == 'V')]
B_seq = seq_light[np.where(seq_filters == 'B')]
I_seq = seq_light[np.where(seq_filters == 'I')]

### Master Dark:

In [117]:
master_V_dark = np.median(seq_dark, axis=0)

### Master Flat:

In [118]:
comb_V_flat = np.median(V_flats,axis=0) - master_V_dark
div = master_V_dark/np.median(comb_V_flat)
master_V_flat = comb_V_flat - div

### Subtract dark and divide by flat for each science frame

In [119]:
V_proc = []

for i in range(len(V_seq)):
    V_proc_temp = V_seq[i] - master_V_dark
    V_proc.append(V_proc_temp/master_V_flat)

### Shifting frames

In [120]:
V1 = V_seq[0]
V2 = np.roll(V_proc[1], [0,4], axis=(0,1))
V3 = np.roll(V_proc[2], [0,7], axis=(0,1))
V4 = np.roll(V_proc[3], [0,7], axis=(0,1))
V5 = np.roll(V_proc[4], [-1,8], axis=(0,1))
V6 = np.roll(V_proc[5], [-2,7], axis=(0,1))
V7 = np.roll(V_proc[6], [-3,5], axis=(0,1))
V8 = np.roll(V_proc[7], [-6,3], axis=(0,1))
V9 = np.roll(V_proc[8], [-8,2], axis=(0,1))
V10 = np.roll(V_proc[9], [-10,-1], axis=(0,1))

V = [V1, V2, V3, V4, V5, V6, V7, V8, V9, V10]
V = np.asarray(V)
V = np.median(V, axis=0)
Vf = np.flip(V, 0)

## PSF characteristics
Use the image in one band you have selected to measure the typical FWHM (in pixels) of the stars in the field.  You should check several stars (3-5).  They should be isolated and relatively bright, but be sure to avoid saturated stars.  Record the FWHM values.

This part is from stackoverflow

In [121]:
from astropy.modeling import models, fitting

In [122]:
# Cut out smaller box around PSF
def cutout(image, bb, center):
    xc = int(center[0])
    yc = int(center[1])
    box = image[yc-bb:yc+bb, xc-bb:xc+bb]
    xp, yp = box.shape
    return xp, yp, box

In [123]:
def gaussian(yp, xp, box):
    # Generate grid of same size like box to put the fit on
    y, x, = np.mgrid[:yp, :xp]
    # Declare what function you want to fit to your data
    f_init = models.Gaussian2D()
    # Declare what fitting function you want to use
    fit_f = fitting.LevMarLSQFitter()
    
    # Fit the model to your data (box)
    fV = fit_f(f_init, x, y, box)
    
    return fV, x, y

In [124]:
def sigma_avg(x,y):
    return np.mean([float(x), float(y)])

### Using astrometry.net I identified a bunch of stars and calculated the plate scale to be:

## 0.605 arcsec/pixel ##

http://nova.astrometry.net/user_images/4128020#annotated

In [127]:
def psf(image, targ_loc, boxsize, apt, apt_sky_in, apt_sky_out, plot='star'): 
    
    # Cut out a square of 'radius' boxsize around a star (specified by targ_loc)
    F_sky_pix = []
    targ_loc[1] = 1266-targ_loc[1]
    xp, yp, current_image = cutout(image, boxsize, targ_loc)
    
    # Fit a 2D gaussian to the cutout
    fV, x, y = gaussian(xp, yp, current_image)

###### Use this to debug
#    if plot=='star':
#        plt.imshow(current_image)
#        plt.show()
#    elif plot =='gaussian':
#        plt.imshow(fV(x,y))
#        plt.show()
#    if plot=='cutout':
        #plt.imshow(circ_image)
        #plt.show()
    
    # Sneakily convert 2D Gaussian into 1D by taking the average of the two sigmas.
    sigma = sigma_avg(fV.x_stddev[0],fV.y_stddev[0])
    fwhm = 2.355*float(sigma)
    fit = fV(x, y)
    # Find the center of the star based on the center of the fitted 2D Gaussian
    center = np.where(fit == np.max(fit))
 
    # Cut out a circle and a 'donut' based on the specified parameters.
    # Calculate number of pixels in the circle
    circ_image = np.zeros(current_image.shape)
    N_targ = 0
    for i in range(len(current_image)):
        for j in range(len(current_image[0])):
            dist = np.sqrt((center[0]-i)**2+(center[1]-j)**2)
            if dist >= apt_sky_in*fwhm and dist <= apt_sky_out*fwhm:
                F_sky_pix.append(current_image[i][j])  
            if (dist > apt*fwhm):
                circ_image[i][j] = 0.0 # this is the same cutout image but with all values outside of the radius set to NaN
            else:
                circ_image[i][j] = current_image[i][j]
                N_targ+=1 # total number of pixels in the 'aperture'

    # Total flux in the aperture, median flux of the sky in the donut
    F_sky_pix_med = np.median(F_sky_pix)
    F_targ_raw = 0
    F_targ_raw = np.sum(circ_image)
       
    # Get flux per second (5 second exposure)
    F_targ = F_targ_raw - N_targ*F_sky_pix_med
    F_targ_pers = F_targ/5. #ADU/s
                        
    return F_targ_pers

In [139]:
def get_flux_per_s(ap, apsky_in, apsky_out):
    tnames = ['2-3694-3626-1', '2-3694-2565-1', '2-3694-2943-1', '2-3694-1712-1', '2-3694-2537-1', '2-3694-2413-1', '2-3694-3750-1', '2-3694-3451-1', '2-3694-1921-1', '2-3694-1822-1', '2-3694-1719-1',  '2-3694-1853-1', '2-3694-1363-1']
    targlocs = [[440, 134], [382,1013], [529, 634], [892, 1139], [334, 462], [506, 139], [194, 272], [329,956], [613,642], [1150, 146], [1032, 812], [1463, 134], [1485, 1077]]
    # Sometimes things don't work. I blame Gaussian fitter. That's why I kept changing boxseizes for each star
    # individually until it worked. I guess it has something to do with not having enough background for a fit
    boxsizes = [25, 35, 45, 30, 26, 35, 36, 40, 40, 40, 27, 20, 56]
    Flux_per_s = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    for i in range(len(targlocs)):
        try:
            F_targ_pers = psf(Vf, targlocs[i], boxsizes[i], ap, apsky_in, apsky_out, plot='cutout')
            Flux_per_s[i] = F_targ_pers
        except TypeError:
            print(i+1, 'TypeError')
        except ValueError:
            print(i+1, 'ValueError')
    return Flux_per_s

### magnitudes of the selected stars from simbad
The stars were identified using nova.astrometry.net. All selected stars were selected to be a considarable distance from other stars.

In [129]:
simbad_mags = [7.7866, 8.2215, 8.3765, 10.17, 9.4150, 8.3904, 10.219, 10.6020, 8.5262, 11.1, 8.4634, 9.3223, 11.320]

In [140]:
def m_i(flux):
    return -2.5*np.log10(flux)

def dm_std(m_cat_std, flux_std):
    dm = m_cat_std - m_i(flux_std)
    return dm

def m_obs(dm, flux_targ):
    m_obs = m_i(flux_targ) + dm
    return m_obs

In [143]:
# Automate everything
def Our_Sim(Flux_per_s):
    simbad_mags = [7.7866, 8.2215, 8.3765, 10.17, 9.4150, 8.3904, 10.219, 10.6020, 8.5262, 11.1, 8.4634, 9.3223, 11.320]
    dm = dm_std(simbad_mags, Flux_per_s)
    DM = np.mean(dm)
    Obs_Mag = m_obs(DM, Flux_per_s)
    Our_vs_Sim = simbad_mags - Obs_Mag
    return Our_vs_Sim

## Simbad - calibrated magnitude for four configurations
The three inputs are: inner apperature, sky 'donut' inner radius and outter radius. \
The calibration process happens for every star (dm is calculated for each star) and then the average dm is taken. This is then used to calibrate observed fluxes and compare them
to simbad magnitudes.

In [135]:
Our_Sim(get_flux_per_s(1.3,2,3))

array([-0.04616713, -0.60101187, -0.68912206,  0.95413035,  0.33434125,
       -0.54353479,  0.48617209,  0.39709701, -0.75009817,  0.88786966,
       -0.59421783, -0.2091124 ,  0.3736539 ])

In [136]:
Our_Sim(get_flux_per_s(0.5,2,3))

array([-0.10000446, -0.57529161, -0.69612783,  0.98194785,  0.41530491,
       -0.57662707,  0.51843585,  0.42075351, -0.73527908,  0.89389975,
       -0.61903256, -0.27238372,  0.34440446])

In [108]:
Our_Sim(get_flux_per_s(2,2,3))

array([-0.0442743 , -0.60406424, -0.68162472,  0.94928422,  0.32810494,
       -0.54643671,  0.4831887 ,  0.39597526, -0.74590035,  0.89849365,
       -0.58942092, -0.22060098,  0.37727546])

In [109]:
Our_Sim(get_flux_per_s(1.3,1.5,3))

array([-0.04434767, -0.60051344, -0.68831321,  0.95414702,  0.33436772,
       -0.54258036,  0.48592915,  0.39606743, -0.75012969,  0.88672121,
       -0.59312296, -0.2102859 ,  0.37206071])